<a href="https://colab.research.google.com/github/mahalakruth/SAJRA/blob/main/SUMMARIZATION_Py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("philschmid/tf-distilbart-cnn-12-6")
model = AutoModelForSeq2SeqLM.from_pretrained("philschmid/tf-distilbart-cnn-12-6", from_tf=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.85k [00:00<?, ?B/s]

tf_model.h5:   0%|          | 0.00/1.22G [00:00<?, ?B/s]

All TF 2.0 model weights were used when initializing BartForConditionalGeneration.

Some weights of BartForConditionalGeneration were not initialized from the TF 2.0 model and are newly initialized: ['model.encoder.embed_tokens.weight', 'model.decoder.embed_tokens.weight', 'lm_head.weight', 'lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 5.5 MB/s eta 0:00:00


In [4]:
import PyPDF2


def extract_text_from_pdf(pdf_path):
    """
    Extract text from a PDF file.
    """
    text = ""
    with open(pdf_path, "rb") as file:
        reader = PyPDF2.PdfReader(file)
        for page_num in range(len(reader.pages)):
            text += reader.pages[page_num].extract_text()
    return text


In [5]:
def generate_summary(document):
    """
    Generate a summary from the input document.
    """
    # Tokenize the input document
    inputs = tokenizer([document], max_length=1024, return_tensors='pt', truncation=True)
    # Generate summary
    summary_ids = model.generate(inputs['input_ids'], num_beams=4, min_length=100, max_length=200, early_stopping=True)
    # Decode the summary
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

In [6]:

def process_document(pdf_path):
    """
    Process a document: extract text and generate summary.
    """
    # Extract text from the PDF
    document = extract_text_from_pdf(pdf_path)
    # Generate summary
    summary = generate_summary(document)
    return document, summary

In [7]:
def main():
    """
    Main function to process multiple documents.
    """
    documents = []
    summaries = []
    pdf_paths = ["/content/Doc2.pdf","/content/Software _Engineer.pdf","/content/abila.pdf"]  # Add paths to your documents

    for pdf_path in pdf_paths:
        document, summary = process_document(pdf_path)
        documents.append(document)
        summaries.append(summary)

    # Print summaries
    for i, summary in enumerate(summaries):
        print(f"Summary for Document {i+1}:")
        print(summary)
        print()
        # Example usage:
if __name__ == "__main__":
    main()

Summary for Document 1:
 The Engineers Council for Professional Development defines engineering as the application of science to the optimum conversion of natural resources to the uses of humankind. Engineering is based principally on physics, chemistry, and mathematics. Unlike scientists, engineers are not free to select the problems that interest them. They must solve problems as they arise, and their solutions must satisfy conflicting requirements. Problem solving is common to all engineering work. Of great importance is the process of creative  synthesis or design, putting ideas together to create a new and optimum solution.

Summary for Document 2:
 A software engineer is a person who has technical and creative skills to make software programs. The software developed by them lets users perform some tasks on a computer, including playing a game and entering data into a database. Without a software engineer, you would not have been able to use social media apps, do your study  resea

In [7]:
from transformers import BartForConditionalGeneration
model = BartForConditionalGeneration.from_pretrained("sshleifer/distilbart-cnn-12-6")
print("Number of parameters:", model.num_parameters())

In [8]:
print(len(tokenizer))

50265


In [9]:
print(tokenizer)

BartTokenizerFast(name_or_path='philschmid/tf-distilbart-cnn-12-6', vocab_size=50265, model_max_length=1024, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	1: AddedToken("<pad>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	2: AddedToken("</s>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	3: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=True, special=True),
	50264: AddedToken("<mask>", rstrip=False, lstrip=True, single_word=False, normalized=True, special=True),
}


In [10]:
print(model)

BartForConditionalGeneration(
  (model): BartModel(
    (shared): Embedding(50264, 1024, padding_idx=1)
    (encoder): BartEncoder(
      (embed_tokens): Embedding(50264, 1024, padding_idx=1)
      (embed_positions): BartLearnedPositionalEmbedding(1026, 1024)
      (layers): ModuleList(
        (0-11): 12 x BartEncoderLayer(
          (self_attn): BartAttention(
            (k_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (v_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (q_proj): Linear(in_features=1024, out_features=1024, bias=True)
            (out_proj): Linear(in_features=1024, out_features=1024, bias=True)
          )
          (self_attn_layer_norm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (activation_fn): GELUActivation()
          (fc1): Linear(in_features=1024, out_features=4096, bias=True)
          (fc2): Linear(in_features=4096, out_features=1024, bias=True)
          (final_layer_norm): LayerN